In [6]:
import numpy as np
from ase.io import read
from ase.data import atomic_masses
from calorine.calculators import CPUNEP
from calorine.tools import get_force_constants, relax_structure
import os
import warnings
warnings.filterwarnings("ignore", message="logm result may be inaccurate*")

In [7]:
os.makedirs('DOS', exist_ok=True)

In [8]:
structure_file = "Structure/Si_dot_1.09nm.xyz"
#structure_file = "Structure/Si_dot_1.63nm.xyz"
#structure_file = "Structure/Si_dot_2.17nm.xyz"
#structure_file = "Structure/Si_dot_3.26nm.xyz"
structure = read(structure_file)

positions = structure.get_positions()

# Get atomic masses for each atom in the structure (in atomic mass units)
masses = np.array([atomic_masses[Z] for Z in structure.get_atomic_numbers()])
N = len(masses)

# Attach NEP calculator
calculator = CPUNEP('NEP/Si_2022_NEP3_3body.txt')
structure.calc = calculator

In [9]:
# Relax the atomic positions to minimize forces
relax_structure(structure, fmax=0.0001)

# Get second-order force constants
phonon = get_force_constants(structure, calculator, [1, 1, 1])

In [10]:
fc2 = phonon.force_constants  # Raw force constant matrix (shape: N × N × 3 × 3)
print(fc2.shape)              # Check its shape: (N, N, 3, 3)

(64, 64, 3, 3)


In [11]:
H = np.zeros((3*N, 3*N))
amu_to_kg = 1.66053906660e-27

for i in range(N):
    for j in range(N):
        m_i = masses[i] * amu_to_kg
        m_j = masses[j] * amu_to_kg
        phi_ij = fc2[i, j]  # shape = (3, 3)
        H[3*i:3*i+3, 3*j:3*j+3] = phi_ij / np.sqrt(m_i * m_j)

In [12]:
# Solve the eigenvalue problem
eigenvals, eigenvecs = np.linalg.eigh(H)

In [13]:
eigenvals = np.real(eigenvals) * 16.0218
omega_squared = np.clip(eigenvals, 0, None)
omega = np.sqrt(omega_squared)  # rad/s

# Save discrete frequency spectrum to file
dos_filename = f"DOS/Freq_{os.path.basename(structure_file).replace('.xyz', '')}.txt"
np.savetxt(dos_filename, omega, header='Frequency (rad/s)', fmt='%.6e')

print(f"Frequencies saved to: {dos_filename}")

Frequencies saved to: DOS/Freq_Si_dot_1.09nm.txt
